Anapsky' RecSys

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
from sklearn.metrics.pairwise import cosine_similarity

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [2]:
# Код тут

df_movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv', parse_dates=['release_date'])
df_credits = pd.read_csv('../../datasets/tmdb_5000_credits.csv')

df = pd.merge(df_movies, df_credits, how='left', left_on='id', right_on='movie_id')
df = df[df['status'] == 'Released']
df = df.dropna(subset=['overview', 'genres', 'keywords'])

print(f'Количество фильмов, оставшихся в выборке: {len(df)}.')

Количество фильмов, оставшихся в выборке: 4792.


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [3]:
# Код тут

df['keywords'] = df['keywords'].apply(lambda x: " ".join([str['name'] for str in ast.literal_eval(x)]))

df['overview_keywords'] = df['overview'] + ' ' + df['keywords']

df['overview_keywords'] = df['overview_keywords'].fillna('')

df['date'] = df['release_date'].dt.year

df['genres_filtered'] = df['genres'].apply(lambda x: " ".join([str['name'] for str in ast.literal_eval(x)]))

vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')

tfidf_matrix = vectorizer.fit_transform(df['overview_keywords'])

print(f"Размер получившейся матрицы: {tfidf_matrix.shape}.")

Размер получившейся матрицы: (4792, 10000).


## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [4]:
cosine_similarity(tfidf_matrix)

array([[1.        , 0.00669786, 0.        , ..., 0.0049405 , 0.01317786,
        0.        ],
       [0.00669786, 1.        , 0.        , ..., 0.019802  , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01172881, 0.        ,
        0.        ],
       ...,
       [0.0049405 , 0.019802  , 0.01172881, ..., 1.        , 0.01716662,
        0.02458099],
       [0.01317786, 0.        , 0.        , ..., 0.01716662, 1.        ,
        0.01253005],
       [0.        , 0.        , 0.        , ..., 0.02458099, 0.01253005,
        1.        ]])

In [5]:
# df_distance.head(5)

NameError: name 'df_distance' is not defined

In [6]:
# Код тут

df_distance = pd.DataFrame(cosine_similarity(tfidf_matrix), index=df['id'], columns=df['id'])

df_distance.to_csv('../assets/distance.csv')
df.to_csv('../assets/movies.csv')

In [7]:
df_distance.head(2)

id,19995,285,206647,49026,49529,559,38757,99861,767,209112,...,182291,286939,124606,14337,67238,9367,72766,231617,126186,25975
id,,,,,,,,,,,,,,,,,,,,,
19995,1.000000,0.006698,0.0,0.011519,0.193628,0.043216,0.008125,0.048032,0.0,0.012418,...,0.0,0.0,0.017703,0.033747,0.0,0.000000,0.0,0.004940,0.013178,0.0
285,0.006698,1.000000,0.0,0.014123,0.040554,0.023146,0.004609,0.014998,0.0,0.000000,...,0.0,0.0,0.014527,0.000000,0.0,0.009755,0.0,0.019802,0.000000,0.0


In [8]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,title_x,vote_average,vote_count,movie_id,title_y,cast,crew,overview_keywords,date,genres_filtered
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...",2009.0,Action Adventure Fantasy Science Fiction
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...",2007.0,Adventure Fantasy Action
